In [24]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

In [25]:
with open('data_20240723.txt', 'r') as f:
    data = f.read()

data_pattern = re.compile(r'(\d{4})\s+(\d{3})\s+(\d{1,2})\s+(\d{1,2})\s+(\d{3})\s+([-]?\d+)\s+(\d+)\s+(\d{3}\.\d)')
matches = data_pattern.findall(data)

In [26]:
columns = ['YEAR', 'DOY', 'HR', 'Kp_index', 'R', 'Dst_index', 'ap_index', 'f10.7_index']
df = pd.DataFrame(matches, columns=columns)

In [27]:
df = df.astype({
    'YEAR': int,
    'DOY': int,
    'HR': int,
    'Kp_index': int,
    'R': int,
    'Dst_index': int,
    'ap_index': int,
    'f10.7_index': float
})


In [28]:
df.head()

,YEAR,DOY,HR,Kp_index,R,Dst_index,ap_index,f10.7_index
0,2024,191,0,20,999,7,7,184.2
1,2024,191,1,20,999,9,7,184.2
2,2024,191,2,20,999,10,7,184.2
3,2024,191,3,17,999,7,6,184.2
4,2024,191,4,17,999,4,6,184.2


In [29]:
df.shape

(24, 8)

In [73]:
new_df = df[['YEAR', 'R', 'f10.7_index']] 
new_df['TEC'] = np.random.uniform(7, 40, size=len(new_df))
print(new_df.head())

   YEAR    R  f10.7_index        TEC
0  2024  999        184.2  14.568092
1  2024  999        184.2  19.529626
2  2024  999        184.2  20.876190
3  2024  999        184.2  20.838248
4  2024  999        184.2  22.188218


C:\Users\Admin\AppData\Local\Temp\ipykernel_3264\598011330.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['TEC'] = np.random.uniform(7, 40, size=len(new_df))


In [74]:
new_df.shape

(24, 4)

In [75]:
def df_to_X_y2(df, window_size=6):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i:i + window_size]]
        X.append(row)
        label = df_as_np[i + window_size][0]  # Adjust the label indexing as necessary
        y.append(label)
    return np.array(X), np.array(y)

NEW_WINDOW_SIZE = 6
X_new, y_new = df_to_X_y2(new_df, NEW_WINDOW_SIZE)




In [76]:
scaler = MinMaxScaler()

# Fit and transform the new data
X_new_normalized = scaler.fit_transform(X_new.reshape(-1, X_new.shape[-1])).reshape(X_new.shape)
y_new_normalized = scaler.fit_transform(y_new.reshape(-1, y_new.shape[-1])).reshape(y_new.shape)

print(X_new_normalized.shape, y_new_normalized.shape)

(18, 6, 4) (18,)


In [77]:
# Load the trained model
model4 = load_model('models/model4/m4.keras')


In [80]:
# Predicting the values
predictions = model4.predict(X_new_normalized)

# Inverse transform the predictions if necessary
#predictions = scaler.inverse_transform(predictions)

print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[[0.06502514]
 [0.06191748]
 [0.07720579]
 [0.07298747]
 [0.06968622]
 [0.0598887 ]
 [0.06996785]
 [0.05573836]
 [0.05986166]
 [0.07884497]
 [0.07012325]
 [0.07476675]
 [0.06861025]
 [0.06897419]
 [0.05893276]
 [0.06304539]
 [0.05702827]
 [0.07277729]]
